In [1]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\a1831\Desktop\NBS_project


In [2]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import src.model as model
import numpy as np

CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 41
MDs = create_MDs(config, seed=seed)
# np.random.shuffle(MDs)  # Shuffle the MDs

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 0.1, 'l': 0.8, 'cn': 0.900369449497976, 'Fn': 1.184383282690447, 'kn': 0.6414529928884467, 'omega_n': 1, 'Rn': np.float64(4.13472934445215)}
{'s': 0.1, 'l': 0.8, 'cn': 1.0415462755795253, 'Fn': 1.7637226511189565, 'kn': 0.6348125738082766, 'omega_n': 1, 'Rn': np.float64(10.248845012378245)}
{'s': 0.1, 'l': 0.8, 'cn': 0.932903940019097, 'Fn': 2.1321345406869066, 'kn': 0.24902581391857953, 'omega_n': 1, 'Rn': np.float64(6.716553285779903)}
{'s': 0.1, 'l': 0.8, 'cn': 0.8278081871246422, 'Fn': 3.8199302863134226, 'kn': 0.35174154718363215, 'omega_n': 1, 'Rn': np.float64(9.231363952354403)}
{'s': 0.1, 'l': 0.8, 'cn': 1.0432905838092195, 'Fn': 3.913825949541608, 'kn': 0.43740643005316393, 'omega_n': 1, 'Rn': np.float64(7.948101841125762)}
{'s': 0.1, 'l': 0.8, 'cn': 0.9766658416311175, 'Fn': 2.4920855793255217, 'kn': 0.5668848317824697, 'omega_n': 1, 'Rn': np.float64(3.859406010253179)}
{'s': 0.1, 'l': 0.8, 'cn': 0.833279172556006, 'Fn': 1.5048957788256199, 'kn': 0.3583132239990442, 'om

In [3]:
T = 100
N = len(MDs)
lam_uni, p_uni, r_uni, Dmax_uni = np.zeros(N),np.zeros(N),np.zeros(N),0
for _ in range(T):
    l,p,r,d = baseline.uniform_baseline(esp, MDs,43)
    lam_uni += l
    p_uni += p 
    r_uni += r
    Dmax_uni += d
lam_uni, p_uni, r_uni, Dmax_uni = lam_uni/T, p_uni/T, r_uni/T, Dmax_uni/T

print(f"Uniform λ: {lam_uni}")
print(f"Uniform p: {p_uni}")  
print(f"Uniform Dmax: {Dmax_uni}")
print(f"Uniform r: {r_uni}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("sum lambda:", np.sum(lam_uni), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_uni[i],lam_uni[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_uni, " D0:", esp.D0)
max_Dn = max(md.delay(p_uni[i], lam_uni[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_uni == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_uni[i], lam_uni[i]) <= Dmax_uni for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_uni <= esp.D0)
print("pn<=Fn:", all(p_uni[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_uni[i] <= p_uni[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_uni), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_uni[i] >= md.Ln(p_uni[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_uni[i]) for i,md in enumerate(MDs)]))

Uniform λ: [1.66666667 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667
 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667
 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667
 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667
 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667]
Uniform p: [0.84743792 0.28195589 0.252951   3.24017574 2.38647269 0.72703071
 1.18460245 1.03547707 1.75634033 2.05964105 4.40729725 1.47262306
 1.44896219 0.7298894  1.48821631 0.22344321 0.68972057 4.12029005
 2.52400625 2.52986244 1.28105471 3.08535723 1.46580568 3.16661375
 1.37577399 0.35654026 0.606453   0.85719577 0.3429618  1.495131  ]
Uniform Dmax: 0.8997265161208552
Uniform r: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
social welfare: -57.227030987145255
sum lambda: 49.99999999999997  lambda0: 50
['MD 1 Dn: 0.13621378869116738', 'MD 2 Dn: 0.5480335053780364', 'MD 3 Dn: 0.68368611875298

In [4]:
T = 100
N = len(MDs)
lam_pro, p_pro, r_pro, Dmax_pro = np.zeros(N),np.zeros(N),np.zeros(N),0
for _ in range(T):
    l,p,r,d = baseline.proportional_baseline(esp,MDs,43)
    lam_pro += l
    p_pro += p 
    r_pro += r
    Dmax_pro += d
lam_pro, p_pro, r_pro, Dmax_pro = lam_pro/T, p_pro/T, r_pro/T, Dmax_pro/T
print(f"Proportional λ: {lam_pro}")
print(f"Proportional p: {p_pro}")  
print(f"Proportional Dmax: {Dmax_pro}")
print(f"Proportional r: {r_pro}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("sum lambda:", np.sum(lam_pro), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_pro[i],lam_pro[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_pro, " D0:", esp.D0)
max_Dn = max(md.delay(p_pro[i], lam_pro[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_pro == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_pro[i], lam_pro[i]) <= Dmax_pro for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_pro <= esp.D0)
print("pn<=Fn:", all(p_pro[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_pro[i] <= p_pro[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_pro), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_pro[i] >= md.Ln(p_pro[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_pro[i]) for i,md in enumerate(MDs)]))

Proportional λ: [0.67714464 1.00836897 1.21900022 2.18395968 2.23764243 1.42479418
 0.86039049 1.9032233  2.22173503 2.4613594  2.63190918 0.92914221
 2.65626163 1.18079321 1.84843927 1.40466885 1.00268707 2.46887966
 1.69786546 2.59591542 1.37871542 1.85929056 1.32879298 1.91271085
 0.78797722 2.53134954 2.01157116 0.60511309 0.70558047 2.26471841]
Proportional p: [0.81991331 0.23159745 0.21785501 3.24682968 2.4053312  0.71202891
 1.16859253 1.04941135 1.78209649 2.09454196 4.41074783 1.46624505
 1.50607108 0.70184612 1.49662148 0.20256885 0.65301173 4.12338633
 2.52441007 2.56442096 1.26920237 3.08620282 1.45480922 3.16773831
 1.37562508 0.42337924 0.63077399 0.83691575 0.27577322 1.52661997]
Proportional Dmax: 0.89888523904807
Proportional r: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
social welfare: -57.60299996894331
sum lambda: 50.00000000000003  lambda0: 50
['MD 1 Dn: 0.12865924591232628', 'MD 2 Dn: 0.539811507909453', 'MD 3 Dn: 

In [5]:
lam_non, p_non, r_non, Dmax_non = baseline.non_cooperative_baseline(esp,MDs)
print(f"Non-cooperative λ: {lam_non}")
print(f"Non-cooperative p: {p_non}")  
print(f"Non-cooperative Dmax: {Dmax_non}")
print(f"Non-cooperative r: {r_non}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_non, p_non))
print("sum lambda:", np.sum(lam_non), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_non[i],lam_non[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_non, " D0:", esp.D0)
max_Dn = max(md.delay(p_non[i], lam_non[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_non == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_non[i], lam_non[i]) <= Dmax_non for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_non <= esp.D0)
print("pn<=Fn:", all(p_non[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_non[i] <= p_non[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_non), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_non[i] >= md.Ln(p_non[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_non[i]) for i,md in enumerate(MDs)]))

Non-cooperative λ: [7.99174525e-02 4.61719202e-01 3.20321809e+00 3.86119754e+00
 2.39556042e+00 1.43269603e+00 2.55555202e+00 7.40355728e-02
 3.08841213e+00 2.66716266e-01 1.88328136e+00 6.35330246e-01
 3.70677762e-01 2.49612744e+00 2.61192607e+00 1.03343401e+00
 1.78244566e+00 1.23903690e+00 2.13318905e+00 3.43908457e+00
 2.13990207e+00 1.22237023e+00 2.35106481e+00 3.82105840e+00
 9.70341164e-01 7.01289119e-01 3.93382238e-15 2.11264308e-01
 1.67336120e+00 1.86579088e+00]
Non-cooperative p: [0.1177335  0.14612471 0.36620061 0.41824054 0.30124682 0.22621899
 0.31425935 0.11514187 0.35660001 0.13166472 0.25999271 0.1600859
 0.13875579 0.30924832 0.31844224 0.19217947 0.25168034 0.20904527
 0.28136592 0.38658844 0.28085371 0.20705138 0.29752387 0.41557629
 0.18769837 0.16557611 0.11035653 0.12922768 0.24342617 0.25873085]
Non-cooperative Dmax: 0.749216419875129
Non-cooperative r: [0.09602722 0.119184   0.29868497 0.34113041 0.24570657 0.18451146
 0.25632001 0.0939134  0.29085441 0.107389

In [7]:
lam_con, p_con, r_con, Dmax_con = baseline.contract_baseline(esp,MDs)
print(f"Contract λ: {lam_con}")
print(f"Contract p: {p_con}")  
print(f"Contract Dmax: {Dmax_con}")
print(f"Contract r: {r_con}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_con, p_con))
print("sum lambda:", np.sum(lam_con), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_con[i],lam_con[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_con, " D0:", esp.D0)
max_Dn = max(md.delay(p_con[i], lam_con[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_con == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_con[i], lam_con[i]) <= Dmax_con for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_con <= esp.D0)
print("pn<=Fn:", all(p_con[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_con[i] <= p_con[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_con), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_con[i] >= md.Ln(p_con[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_con[i]) for i,md in enumerate(MDs)]))

c:\Users\a1831\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\optimize\_differentiable_functions.py:376: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
c:\Users\a1831\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\optimize\_differentiable_functions.py:728: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


Contract λ: [0.07040282 0.41360898 3.21206686 3.99745842 2.34934275 1.39561969
 2.62845431 0.181711   3.1237042  0.35376252 1.88117539 0.68383896
 0.28196488 2.55233737 2.61906142 1.13701582 1.89795562 1.3615803
 2.11934027 3.35743829 2.07013679 1.28798813 2.24765548 3.93509769
 0.76916064 0.64387313 0.00634629 0.04713369 1.65199428 1.72277399]
Contract p: [0.20330088 0.22489052 0.45083666 0.51203956 0.38089188 0.31007488
 0.40380438 0.20642794 0.44264209 0.22320678 0.34269804 0.24672
 0.21412628 0.39701092 0.4021563  0.28364031 0.34337658 0.30274333
 0.36551724 0.46663064 0.35871256 0.29505725 0.37231281 0.50853984
 0.25574925 0.24410342 0.19503481 0.20414702 0.32497857 0.33039749]
Contract Dmax: 0.5409113003123949
Contract r: [0.16405481 0.17159927 0.2589836  0.29613785 0.23091724 0.21566372
 0.25848923 0.16894305 0.26007917 0.17937435 0.22567654 0.18950803
 0.16325553 0.25211194 0.24492243 0.2124873  0.23980518 0.22385151
 0.23538968 0.24936597 0.22176245 0.21430296 0.21864961 0.293

In [8]:
lam_swm, p_swm, Dmax_swm = baseline.social_welfare_maximization(esp, MDs)
print(f"SWM λ: {lam_swm}")
print(f"SWM p: {p_swm}")  
print(f"SWM Dmax: {Dmax_swm}")
r_swm = baseline.solve_r_NBP(esp, MDs, Dmax_swm, lam_swm, p_swm)
print(f"SWM r: {r_swm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("sum lambda:", np.sum(lam_swm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_swm[i],lam_swm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_swm, " D0:", esp.D0)
max_Dn = max(md.delay(p_swm[i], lam_swm[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_swm == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_swm[i], lam_swm[i]) <= Dmax_swm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_swm <= esp.D0)
print("pn<=Fn:", all(p_swm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_swm[i] <= p_swm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_swm), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_swm[i] >= md.Ln(p_swm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_swm[i]) for i,md in enumerate(MDs)]))

SWM λ: [0.78311073 0.96306567 2.52486452 2.85520743 2.07851624 1.53370166
 2.18821351 0.86753167 2.46867776 0.96351821 1.80494354 1.1356518
 0.87738886 2.15523414 2.20989988 1.38655276 1.79175739 1.50523346
 1.93449451 2.63466268 1.9261743  1.46865043 2.04002354 2.8318624
 1.15983798 1.10118331 0.66583453 0.72560107 1.67899439 1.73961164]
SWM p: [0.23950535 0.24843502 0.37528402 0.40020215 0.33871445 0.30022978
 0.34802834 0.24087226 0.36973852 0.25135062 0.31614449 0.26242347
 0.24136615 0.34474136 0.34893837 0.28311238 0.3144733  0.29366079
 0.33000486 0.38794205 0.32667731 0.28906908 0.33522372 0.39971345
 0.26640576 0.26020285 0.2276099  0.23745252 0.30664001 0.31126352]
SWM Dmax: 0.4765305235523106
SWM r: [2.96245693 2.95643887 2.94865644 2.95419055 2.95064023 2.96624613
 2.96548323 2.96529538 2.95197148 2.9689793  2.96001325 2.96603679
 2.95207385 2.96235566 2.95443813 2.9723819  2.97180886 2.97518061
 2.96121437 2.93665617 2.95095792 2.96871791 2.94319473 2.95390526
 2.94565308 

In [ ]:
lam_opt, p_opt, r_opt, Dmax_opt = baseline.optimal_NBP(esp, MDs)
print(f"Optimal λ: {lam_opt}")
print(f"Optimal p: {p_opt}")  
print(f"Optimal r: {r_opt}")
print(f"Optimal Dmax: {Dmax_opt}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("sum lambda:", np.sum(lam_opt), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_opt[i],lam_opt[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_opt, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_opt == max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_opt[i], lam_opt[i]) <= Dmax_opt for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_opt <= esp.D0)
print("pn<=Fn:", all(p_opt[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_opt[i] <= p_opt[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_opt), "Q_star:", esp.Q(Dmax_opt))
print("rn>=Ln:", all(r_opt[i] >= md.Ln(p_opt[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_opt[i]) for i,md in enumerate(MDs)]))

In [9]:
lamb_admm, p_admm, Dmax_admm = NBS.ADMM(esp, MDs)
print(f"ADMM λ: {lamb_admm}")
print(f"ADMM p: {p_admm}")  
print(f"ADMM Dmax: {Dmax_admm}")
r_admm = NBS.negotiation(esp, MDs,lamb_admm, p_admm, Dmax_admm)
print(f"ADMM r: {r_admm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("sum lambda:", np.sum(lamb_admm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_admm[i],lamb_admm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_admm, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_admm == max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_admm[i], lamb_admm[i]) <= Dmax_admm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_admm <= esp.D0)
print("pn<=Fn:", all(p_admm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lamb_admm[i] <= p_admm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_admm), "Q_star:", esp.Q(Dmax_admm))
print("rn>=Ln:", all(r_admm[i] >= md.Ln(p_admm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_admm[i]) for i,md in enumerate(MDs)]))

ADMM λ: [1.07010069 1.18525609 2.24524765 2.44689047 1.95044944 1.57944056
 2.01637771 1.13693844 2.20613277 1.20139455 1.76317761 1.31185314
 1.12170213 1.9959694  2.03596841 1.48329738 1.75254244 1.56283889
 1.850347   2.32511683 1.84750031 1.53676591 1.92741677 2.43310705
 1.31029405 1.2808585  1.01366635 1.01131876 1.67780695 1.72022373]
ADMM p: [0.26240504 0.26615601 0.35285898 0.36748191 0.32841426 0.30383099
 0.33422617 0.2623706  0.34867992 0.27032458 0.31274894 0.27646552
 0.26085687 0.3319453  0.33496909 0.29079753 0.31128216 0.29821412
 0.32321649 0.36312014 0.32032854 0.29446426 0.32616063 0.36775725
 0.27838647 0.27452229 0.25537964 0.26024515 0.30649042 0.30965805]
ADMM Dmax: 0.4766778672211501
ADMM r: [2.98630727 2.9740687  2.92822852 2.92667225 2.93971808 2.96826971
 2.95159493 2.98728536 2.93250817 2.98792591 2.95522643 2.97929304
 2.97188981 2.94930451 2.9404558  2.97863427 2.96720946 2.97816867
 2.95333314 2.914648   2.943481   2.9725772  2.93334735 2.92688582
 2.956

In [9]:
# print("Nash Product Opt:",model.nash_product_log(esp, MDs, lam_opt, p_opt, r_opt))
print("Nash Product SWM:",model.nash_product_log(esp, MDs, lam_swm, p_swm, r_swm))
print("Nash Product ADMM:",model.nash_product_log(esp, MDs, lamb_admm, p_admm, r_admm))
print("Nash Product Proportional:",model.nash_product_log(esp, MDs, lam_pro, p_pro, r_pro))
print("Nash Product Uniform:",model.nash_product_log(esp, MDs, lam_uni, p_uni, r_uni))
print("Nash Product Non-cooperative:",model.nash_product_log(esp, MDs, lam_non, p_non, r_non))
print("Nash Product Contract:",model.nash_product_log(esp, MDs, lam_con, p_con, r_con))
print("=============================")
# print("SW Opt:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("SW SWM:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("SW ADMM:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("SW Proportional:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("SW Uniform:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("SW Non-cooperative:",model.social_welfare(esp, MDs, lam_non, p_non))
print("SW Contract:",model.social_welfare(esp, MDs, lam_con, p_con))
print("=============================")
# print("Dmax Opt:", Dmax_opt)
print("Dmax SWM:", Dmax_swm)
print("Dmax ADMM:", Dmax_admm)
print("Dmax Proportional:", Dmax_pro)
print("Dmax Uniform:", Dmax_uni)
print("Dmax Non-cooperative:", Dmax_non)
print("Dmax Contract:", Dmax_con)

Nash Product SWM: 33.87869366086002
Nash Product ADMM: 33.86067989729551
Nash Product Proportional: nan
Nash Product Uniform: nan
Nash Product Non-cooperative: -86.52383586523801
Nash Product Contract: -inf
SW SWM: 88.33384951949053
SW ADMM: 88.2841377290702
SW Proportional: -49.979860581700706
SW Uniform: -50.85816304968958
SW Non-cooperative: 84.1729598354875
SW Contract: 97.0
Dmax SWM: 0.4765305235523106
Dmax ADMM: 0.4766778672211501
Dmax Proportional: 0.9779738123526548
Dmax Uniform: 0.9781732634978643
Dmax Non-cooperative: 0.749216419875129
Dmax Contract: 0


c:\Users\a1831\Desktop\NBS_project\src\model.py:11: RuntimeWarning: invalid value encountered in log
  n0 = ESP.omega_0 * np.log(ESP.Q(Dmax)-np.sum(r))
c:\Users\a1831\Desktop\NBS_project\src\model.py:12: RuntimeWarning: invalid value encountered in log
  ns = [md.omega_n*np.log(rn-md.Ln(pn)) for (md,rn,pn) in zip(MDs,r,p)]
c:\Users\a1831\Desktop\NBS_project\src\model.py:12: RuntimeWarning: divide by zero encountered in log
  ns = [md.omega_n*np.log(rn-md.Ln(pn)) for (md,rn,pn) in zip(MDs,r,p)]
